## Preparation

In [ ]:
pip install pmdarima

     |████████████████████████████████| 1.4 MB 4.1 MB/s 
     |████████████████████████████████| 9.8 MB 29.0 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.rcParams["font.family"] = "serif"
% matplotlib inline
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Waseda/4.3 Inaba Zemi/Senior Thesis/Data/"
df = pd.read_csv(path +'df_master.csv')
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df = df.drop(columns={'Unnamed: 0', 'date'})

df_norm = pd.read_csv(path +'df_master_normalized.csv')
df_norm.index = df_norm['date']
df_norm = df_norm.drop(columns={'Unnamed: 0', 'date'})

Mounted at /content/drive


In [ ]:
df

,hospitalized,light-mid_symptoms,severe_symptoms,dead,discharged,PCR_positive,PCR_negative,tested_MA(7days),positive_rate,comparisonPreDay,comparisonPreDeclare,comparisonPreSpread
date,,,,,,,,,,,,
2020-05-01,2673.0,2576.0,97.0,6.0,94.0,18.0,289.0,299.7,0.100,-2.7,-61.5,-74.9
2020-05-02,2783.0,2683.0,100.0,15.0,32.0,19.0,280.0,297.1,0.088,-37.8,-76.1,-85.7
2020-05-03,2856.0,2759.0,97.0,4.0,17.0,32.0,446.0,320.7,0.080,-21.0,-81.1,-88.7
2020-05-04,2915.0,2823.0,92.0,5.0,22.0,7.0,260.0,316.3,0.071,10.3,-79.2,-87.5
2020-05-05,2950.0,2857.0,93.0,0.0,23.0,9.0,135.0,318.7,0.073,0.7,-79.0,-87.4
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-26,180.0,178.0,2.0,0.0,27.0,29.0,1952.0,6596.9,0.006,-6.3,105.8,23.4
2021-12-27,202.0,200.0,2.0,0.0,31.0,58.0,4683.0,6462.6,0.007,-20.7,63.2,6.3
2021-12-28,203.0,201.0,2.0,1.0,29.0,47.0,4878.0,6189.3,0.008,0.7,64.4,7.1


## Stationarity

In [ ]:
# Import modules
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

# Load csv
df = pd.read_csv('df_master.csv')

# Define ADF test for multiple significance levels
def ad_test(df, col):
  name = str(col)
  dataset = df[col]
  result = True
  dftest = adfuller(dataset, autolag = 'AIC')
  sig_level = ['10%', '5%', '1%']
  for sig in sig_level:
    if dftest[0] > dftest[4][sig]:
      reject = sig
      result = False
      break
    else:
      pass
  if result == True:
    result_text = 'Pass'
  else:
    result_text = 'Reject at ' + sig
  return name + '　&　' + "{:.4f}".format(dftest[0]) + '　&　' + "{:.4f}".format(dftest[4]['1%']) 
        + '　&　' + "{:.4f}".format(dftest[4]['5%']) + '　&　' + "{:.4f}".format(dftest[4]['10%'])
         + '　&　' + result_text + "\\" +"\n"

# Run the function and store the results in a txt file
file = open("ADF_tabular.txt","w")
for col in df_norm.columns:
  file.writelines(ad_test(df_norm, col))
file.close()

# Take the First Difference for non-stationary processes
## Take the features of non-stationary process
non_stationary_feat = ['hospitalized', 'light-mid_symptoms', 'severe_symptoms', 'dead', 'PCR_negative', 'tested_MA(7days)']
df_copy = df[non_stationary_feat]

## Take the first difference
df_diff = df_copy.diff()
df_diff = df_diff.dropna()

## Redo the ADF test and store the results to a separate file
file2 = open("ADF_tabular2.txt","w")
for col in df_diff.columns:
  file2.writelines(ad_test(df_diff, col))
file2.close()

# Make a single dataframe with all stationary features
stat_cols = []
for col in df.columns:
  if col not in non_stationary_feat:
    stat_cols.append(col)
df_stat = df[stat_cols]
newdata = pd.merge(df_stat, df_diff, on = 'date')
newdata = newdata.reindex(columns = df.columns)

In [ ]:
for col in df_norm.columns:
  print(ad_test(df_norm, col))

In [ ]:
file = open("ADF_tabular.txt","w")
for col in df_norm.columns:
  file.writelines(ad_test(df_norm, col))
file.close()

In [ ]:
## Take the features of non-stationary process
non_stationary_feat = ['hospitalized', 'light-mid_symptoms', 'severe_symptoms', 'dead', 'PCR_negative', 'tested_MA(7days)']
df_copy = df[non_stationary_feat]
## Take the first difference
df_diff = df_copy.diff()
df_diff = df_diff.dropna()
## Redo the ADF test and store the results to a separate file
file2 = open("ADF_tabular2.txt","w")
for col in df_diff.columns:
  file2.writelines(ad_test(df_diff, col))
file2.close()

,hospitalized,light-mid_symptoms,severe_symptoms,dead,PCR_negative,tested_MA(7days)
date,,,,,,
2020-05-01,1.055796,1.079205,0.538351,0.160344,-1.715922,-1.932167
2020-05-02,1.160251,1.186299,0.586442,1.627536,-1.719098,-1.932934
2020-05-03,1.229571,1.262366,0.538351,-0.165698,-1.660515,-1.925973
2020-05-04,1.285597,1.326422,0.458198,-0.002677,-1.726156,-1.927270
2020-05-05,1.318833,1.360452,0.474229,-0.817783,-1.770269,-1.926563
...,...,...,...,...,...,...
2021-12-26,-1.311535,-1.320898,-0.984546,-0.817783,-1.129040,-0.074736
2021-12-27,-1.290644,-1.298879,-0.984546,-0.817783,-0.165256,-0.114349
2021-12-28,-1.289695,-1.297878,-0.984546,-0.654762,-0.096439,-0.194962


In [ ]:
df_diff = df_copy.diff()
df_diff = df_diff.dropna()
file2 = open("ADF_tabular2.txt","w")
for col in df_diff.columns:
  file2.writelines(ad_test(df_diff, col))
file2.close()

In [ ]:
stat_cols = []
for col in df_norm.columns:
  if col not in cols:
    stat_cols.append(col)
df_stat = df[stat_cols]
newdata = pd.merge(df_stat, df_diff, on = 'date')
newdata = newdata.reindex(columns = df_norm.columns)

['discharged',
 'PCR_positive',
 'positive_rate',
 'comparisonPreDay',
 'comparisonPreDeclare',
 'comparisonPreSpread']

In [ ]:
df_stat = df_norm[stat_cols]
newdata = pd.merge(df_stat, df_diff, on = 'date')
newdata = newdata.reindex(columns = df_norm.columns)

In [ ]:
newdata.to_csv('df_master1.csv')
newdata

,hospitalized,light-mid_symptoms,severe_symptoms,dead,discharged,PCR_positive,PCR_negative,tested_MA(7days),positive_rate,comparisonPreDay,comparisonPreDeclare,comparisonPreSpread
date,,,,,,,,,,,,
2020-05-02,0.104455,0.107094,0.048091,1.467192,-0.607995,-0.626379,-0.003176,-0.000767,0.631309,-1.984874,-3.701965,-3.906879
2020-05-03,0.069320,0.076067,-0.048091,-1.793234,-0.623487,-0.606302,0.058582,0.006961,0.469070,-1.133204,-3.875804,-4.078606
2020-05-04,0.056026,0.064056,-0.080152,0.163021,-0.618323,-0.644911,-0.065640,-0.001298,0.286550,0.453538,-3.809745,-4.009915
2020-05-05,0.033236,0.034030,0.016030,-0.815106,-0.617290,-0.641822,-0.044113,0.000708,0.327110,-0.033130,-3.802791,-4.004191
2020-05-06,0.022790,0.026023,-0.032061,0.815106,-0.633814,-0.644911,-0.013057,-0.005516,0.185150,0.108815,-3.778454,-3.981294
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-26,0.001899,0.002002,0.000000,-0.163021,-0.613159,-0.610935,-0.787685,0.016164,-1.031647,-0.387993,2.622306,2.338260
2021-12-27,0.020891,0.022019,0.000000,0.000000,-0.609028,-0.566149,0.963784,-0.039613,-1.011367,-1.117996,1.141196,1.359416
2021-12-28,0.000950,0.001001,0.000000,0.163021,-0.611093,-0.583137,0.068817,-0.080613,-0.991087,-0.033130,1.182917,1.405210


## Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF

In [ ]:
newdf = newdata.copy()
newdf = newdata.drop(columns = {'positive_rate'})

In [ ]:
# Import modules
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF

# Prepare dataframe 
newdf = newdata.copy()
## Drop the objective function
newdf = newdata.drop(columns = {'positive_rate'})

# Define the VIF generator from a given dataframe 
def VIF_operator(df):
  X_variables = df.copy()
  vif_data = pd.DataFrame()
  vif_data["feature"] = df.columns
  vif_data["VIF"] = [VIF(X_variables.values, i) for i in range(len(X_variables.columns))]
  return vif_data

# Run the function for dataframe
vif1 = VIF_operator(newdf)

# Store the results in txt file
file2 = open('VIF_result1.txt', 'w')
for i in range(11):
  text = str(vif1.values[i][0]) + ' & ' + str(vif1.values[i][1]) + " \\" + "\\" + '\n'
  file2.write(text)
file2.close()

# From the results drop identical features
newdata1 = newdf.copy()
newdata1 = newdata1.drop(columns = ['hospitalized', 'comparisonPreDeclare'])
newdata1.index = pd.to_datetime(newdata1.index)

# Recalculate VIF
vif2 = VIF_operator(newdata1)

#Store the results in txt file
file3 = open('VIF_result2.txt', 'w')
for i in range(9):
  text = str(vif2.values[i][0]) + ' & ' + str(vif2.values[i][1]) + " \\" + "\\" + '\n'
  file3.write(text)
file3.close()

# Store the Pre-processed data set to a csv file
## Get the dataframe which includes the objective function
positive = df.loc[df['positive_rate', 'comparisonPreDeclare']]
df_master_final = pd.merge(positive, newdata1, on = 'date')
## Drop unnecessary column
df_master_final = df_master_final.drop(columns={'comparisonPreDeclare'})
## Rename all the columns
df_master_final = df_master_final.rename(columns={'positive_rate':'Positive\_Rate', 'light-mid_symptoms':'Light-Mid\_Symptoms',
                                                  'severe_symptoms':'Severe\_Symptoms', 'dead':'Dead', 'discharged':'Discharged',
                                                  'PCR_positive': 'PCR\_Positive', 'tested_MA(7days)':'Tested\_MA(7days)',
                                                  'comparisonPreDay':'ComparisonPreDay', 'comparisonPreSpread':'ComparisonPreSpread'})
df_master_final.to_csv('df_master_final.csv')

In [ ]:
vif1 = VIF_operator(newdf)

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
file2 = open('VIF_result1.txt', 'w')
for i in range(11):
  text = str(vif1.values[i][0]) + ' & ' + str(vif1.values[i][1]) + " \\" + "\\" + '\n'
  file2.write(text)
file2.close()

ValueError: ignored

In [ ]:
newdata1 = newdf.copy()
newdata1 = newdata1.drop(columns = ['hospitalized', 'comparisonPreDeclare'])
newdata1.index = pd.to_datetime(newdata1.index)
vif2 = VIF_operator(newdata1)

In [ ]:
file3 = open('VIF_result2.txt', 'w')
for i in range(9):
  text = str(vif2.values[i][0]) + ' & ' + str(vif2.values[i][1]) + " \\" + "\\" + '\n'
  file3.write(text)
file3.close()

In [ ]:
positive = df.loc[df['positive_rate', 'comparisonPreDeclare']]

DatetimeIndex(['2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04',
               '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08',
               '2020-05-09', '2020-05-10',
               ...
               '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24',
               '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28',
               '2021-12-29', '2021-12-30'],
              dtype='datetime64[ns]', name='date', length=609, freq=None)

In [ ]:
df_master_final = pd.merge(positive, newdata1, on = 'date')
df_master_final = df_master_final.drop(columns={'comparisonPreDeclare'})
df_master_final = df_master_final.rename(columns={'positive_rate':'Positive\_Rate', 'light-mid_symptoms':'Light-Mid\_Symptoms', 'severe_symptoms':'Severe\_Symptoms', 
                                                  'dead':'Dead', 'discharged':'Discharged', 'PCR_positive': 'PCR\_Positive', 'tested_MA(7days)':'Tested\_MA(7days)',
                                                  'comparisonPreDay':'ComparisonPreDay', 'comparisonPreSpread':'ComparisonPreSpread'})
df_master_final.to_csv('df_master_final.csv')